<a href="https://colab.research.google.com/github/novoforce/Exploring-Pytorch/blob/master/4_autodifferentiation_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic differentiation mechanism
[Autograd Pytorch](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)

Neural networks usually rely on backpropagation to obtain gradients to update network parameters. The process of obtaining gradients is usually a very complicated and error-prone thing.

The deep learning framework can help us automatically complete this gradient calculation.

Pytorch generally uses the **backward method** to achieve this gradient calculation. **The gradient obtained by this method will be stored under the grad attribute of the corresponding independent variable tensor**.

In addition, the **torch.autograd.grad** function can also be called to achieve gradient calculation.

This is Pytorch's automatic differentiation mechanism.


## One, use the backward method to find the derivative

The **backward method** is usually called on a scalar tensor, and the gradient obtained by this method will be stored under the **grad** property of the corresponding independent variable tensor.

If the **called tensor is non-scalar**, a gradient parameter tensor with the same shape as it must be passed in.

**It is equivalent to using the gradient parameter tensor and the calling tensor as a vector dot product, and the obtained scalar result is then backpropagated**.



### 1, Backpropagation of scalar

In [2]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c derivative 

x = torch.tensor(0.0,requires_grad = True) # x needs to be derivated 
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c 

y.backward() #backward method
dy_dx = x.grad
print(dy_dx)

tensor(-2.)


### 2, Non-scalar backpropagation

In [5]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c

x = torch.tensor([[0.0,0.0],[1.0,2.0]],requires_grad = True) # x needs to be derivated 
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c 

gradient = torch.tensor([[1.0,1.0],[1.0,1.0]])

print("x:\n",x)
print("y:\n",y)
y.backward(gradient = gradient)
x_grad = x.grad
print("x_grad:\n",x_grad)

x:
 tensor([[0., 0.],
        [1., 2.]], requires_grad=True)
y:
 tensor([[1., 1.],
        [0., 1.]], grad_fn=<AddBackward0>)
x_grad:
 tensor([[-2., -2.],
        [ 0.,  2.]])


### 3. Non-scalar backpropagation can be realized by scalar backpropagation

In [6]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c

x = torch.tensor([[0.0,0.0],[1.0,2.0]],requires_grad = True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c 

gradient = torch.tensor([[1.0,1.0],[1.0,1.0]])
z = torch.sum(y*gradient)

print("x:",x)
print("y:",y)
z.backward()
x_grad = x.grad
print("x_grad:\n",x_grad)

x: tensor([[0., 0.],
        [1., 2.]], requires_grad=True)
y: tensor([[1., 1.],
        [0., 1.]], grad_fn=<AddBackward0>)
x_grad:
 tensor([[-2., -2.],
        [ 0.,  2.]])


## Second, use the autograd.grad method to find the derivative

In [7]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c derivative

x = torch.tensor(0.0,requires_grad = True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c


# create_graph set to True will allow the creation of higher order derivatives 
dy_dx = torch.autograd.grad(y,x,create_graph=True)[0]
print(dy_dx.data)

# the second derivative
dy2_dx2 = torch.autograd.grad(dy_dx,x)[0] 

print(dy2_dx2.data)

tensor(-2.)
tensor(2.)


In [8]:
import numpy as np 
import torch 

x1 = torch.tensor(1.0,requires_grad = True)
x2 = torch.tensor(2.0,requires_grad = True)

y1 = x1*x2
y2 = x1+x2


# allows simultaneous a plurality of independent variables the derivative 
(dy1_dx1,dy1_dx2) = torch.autograd.grad(outputs=y1,inputs = [x1,x2],retain_graph = True)
print(dy1_dx1,dy1_dx2)

# If there are multiple dependent variables, it is equivalent to summing the gradient results of multiple dependent variables 
(dy12_dx1,dy12_dx2) = torch.autograd.grad(outputs=[y1,y2],inputs = [x1,x2])
print(dy12_dx1,dy12_dx2)

tensor(2.) tensor(1.)
tensor(3.) tensor(2.)


## Three, use automatic differentiation and optimizer to find the minimum

In [9]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c

x = torch.tensor(0.0,requires_grad = True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)

optimizer = torch.optim.SGD(params=[x],lr = 0.01)


def f(x):
    result = a*torch.pow(x,2) + b*x + c 
    return(result)

for i in range(500):
    optimizer.zero_grad()
    y = f(x)
    y.backward()
    optimizer.step()
   
    
print("y=",f(x).data,";","x=",x.data)

y= tensor(0.) ; x= tensor(1.0000)
